In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!ln -s /content/drive/My\ Drive/ /my\drive

ln: failed to create symbolic link '/mydrive/My Drive': Operation not supported


In [9]:
import pandas as pd
from model.LSTM import LSTM_model
from preprocessing.text_preprocessing import Sentences2LemmaWords
from preprocessing.text_preprocessing import GetUnique
from preprocessing.text_preprocessing import setTokenizer
from preprocessing.text_preprocessing import LemmaWords2Seqeunces
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time
from log.logger import Logger

ModuleNotFoundError: ignored

In [11]:
folder_path = '/mydrive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data'
train = pd.read_csv(folder_path + 'train.tsv.zip', sep='\t')
test = pd.read_csv(folder_path + 'test.tsv.zip', sep='\t')

print('train data shape: ', train.shape)
print('test data shape: ', test.shape)
print('----------[train infomation]----------')
print(train.info())
print('----------[test infomation]----------')
print(test.info())

FileNotFoundError: ignored

In [ ]:
train_tokens = Sentences2LemmaWords(train.Phrase)
test_tokens = Sentences2LemmaWords(test.Phrase)

2020-02-17 14:28:18,354[DEBUG|preprocessing, 17] Sentence2LemmaWords: start
/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib64/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


257 / 156060 Phrase

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


10935 / 156060 Phrase

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'log'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


...

155794 / 156060 Phrase

2020-02-17 14:30:23,803[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 125.44864225387573sec
2020-02-17 14:30:23,804[DEBUG|preprocessing, 17] Sentence2LemmaWords: start


156060 / 156060 Phrase
66065 / 66292 Phrase

2020-02-17 14:31:16,587[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 52.7831552028656sec


66292 / 66292 Phrase


In [ ]:
unique_words, maxlen = GetUnique(train_tokens)
print(len(unique_words), maxlen)

2020-02-17 14:31:16,595[DEBUG|preprocessing, 44] GetUnique: start
2020-02-17 14:31:16,716[DEBUG|preprocessing, 55] GetUnique: finish. 계산 시간 0.12096333503723145sec


13745 48


In [ ]:
target = train.Sentiment
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tokens,
    y_target,
    test_size=0.2,
    stratify=y_target)

In [ ]:
tokenizer = setTokenizer(x_train, len(unique_words))
x_train = LemmaWords2Seqeunces(x_train, tokenizer, maxlen)
x_valid = LemmaWords2Seqeunces(x_valid, tokenizer, maxlen)
x_test = LemmaWords2Seqeunces(test_tokens, tokenizer, maxlen)

2020-02-17 14:31:17,664[DEBUG|preprocessing, 62] setTokenizer: start
2020-02-17 14:31:17,800[DEBUG|preprocessing, 69] setTokenizer: finish. 계산 시간 0.13647747039794922sec
2020-02-17 14:31:17,801[DEBUG|preprocessing, 76] LemmaWords2Seqeunces: start
2020-02-17 14:31:18,905[DEBUG|preprocessing, 83] LemmaWords2Seqeunces: finish. 계산 시간 1.1042125225067139sec
2020-02-17 14:31:18,909[DEBUG|preprocessing, 76] LemmaWords2Seqeunces: start
2020-02-17 14:31:19,171[DEBUG|preprocessing, 83] LemmaWords2Seqeunces: finish. 계산 시간 0.26175379753112793sec
2020-02-17 14:31:19,172[DEBUG|preprocessing, 76] LemmaWords2Seqeunces: start
2020-02-17 14:31:19,863[DEBUG|preprocessing, 83] LemmaWords2Seqeunces: finish. 계산 시간 0.6907122135162354sec


In [ ]:
model = LSTM_model(len(unique_words), maxlen, num_classes)

logger = Logger('main')
start_time = time.time()
logger.debug('fit start')

model.fit(x_train,
          y_train,
          validation_data=(x_valid, y_valid),
          epochs=5,
          batch_size=1024,
          verbose=1)

fit_time = time.time() - start_time
logger.debug('fit end. fit_time {}'.format(fit_time))

2020-02-17 14:31:19,880[DEBUG|LSTM_model_generate, 13] LSTM_model: start
2020-02-17 14:31:21,223[DEBUG|LSTM_model_generate, 28] LSTM_model: finish
2020-02-17 14:31:21,224[DEBUG|main, 5] fit start
/usr/local/lib64/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 124848 samples, validate on 31212 samples
Epoch 1/5


ResourceExhaustedError:  OOM when allocating tensor with shape[13745,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node ReadVariableOp_5 (defined at /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_1725]

Function call stack:
keras_scratch_graph


In [ ]:
y_pred = model.predict_classes(x_test)

submission = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': y_pred
})
submission.to_csv(folder_path + 'submission.csv', index=False)